# 1. Install and Import Libraries

In [1]:
!pip install seaborn
!pip install xlrd
#!pip install https://github.com/aboSamoor/pycld2/zipball/e3ac86ed4d4902e912691c1531d0c5645382a726
!python -m pip install -U pycld3

Requirement already up-to-date: pycld3 in /Users/cultureplex/opt/anaconda3/lib/python3.8/site-packages (0.22)


In [2]:
import csv, os
import pandas as pd
import matplotlib
import cld3
from pathlib import Path

# 2. Settings

In [3]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

# 3. Functions

In [4]:
#filter records that are 90% written in english. 
def filter_english_records(df):    
    df['language'] = ""
    for idx, row in df.iterrows():         
        language = ""
        maxx = 0
        for lang in cld3.get_frequent_languages(row['text'],num_langs=3):
            if lang.proportion > maxx:
                language = lang
                maxx = lang.proportion

        if (language):
            if (language.proportion >= 0.9): #90%
                df.loc[idx, "language"] = language.language
            else:            
                df.loc[idx, "language"] = 'multiple'
        else:        
            df.loc[idx, "language"] = 'undefined'
    return df

# 4. Filter english texts

## 4.1 Distribution of languague

In [5]:

data = []
for p in Path('data/').glob('*.txt'):
    data.append([p.name, p.read_text()])
    
df = pd.DataFrame(data, columns = ['title', 'text'])
print(len(df), 'FanFiction stories')
df.head()

2067 FanFiction stories


,title,text
0,The_Start_of_Our_Life_Together_by_Stagelove123...,"\nChapter 1 Hey guys! So, after I wrote the st..."
1,Full_Metal_Stratos_by_Grimm48-g8fp3r2o.txt,\nChapter 1Full Metal StratosDisclaimer: I don...
2,Expect_The_Unexpected_by_sonxathatbruhgirl-nli...,\nChapter One A/N: Hiii guys.So I'm doing a ne...
3,A_Rookie_s_Courage_by_CanadaGrizzly-uxc6kcgd.txt,\nA Rookie's CourageThe wind whipped her hair ...
4,Watashi_koi_shiteru_by_alxslava-q8aodmgc.txt,"\nComprender a una personaWatashi, koi shiteru..."


In [8]:
df = filter_english_records(df)
df.head()

,title,text,language
0,The_Start_of_Our_Life_Together_by_Stagelove123...,"\nChapter 1 Hey guys! So, after I wrote the st...",en
1,Full_Metal_Stratos_by_Grimm48-g8fp3r2o.txt,\nChapter 1Full Metal StratosDisclaimer: I don...,en
2,Expect_The_Unexpected_by_sonxathatbruhgirl-nli...,\nChapter One A/N: Hiii guys.So I'm doing a ne...,en
3,A_Rookie_s_Courage_by_CanadaGrizzly-uxc6kcgd.txt,\nA Rookie's CourageThe wind whipped her hair ...,en
4,Watashi_koi_shiteru_by_alxslava-q8aodmgc.txt,"\nComprender a una personaWatashi, koi shiteru...",es


In [9]:
df.groupby('language').agg({'title':'count'}).sort_values('title', ascending=False)

,title
language,
en,1742
es,145
fr,76
id,53
ms,25
pt,9
de,5
vi,3
zh,2


## 4.3 Filter english records

In [10]:
print('total records', len(df))
df = df[df['language'].str.contains("en")]
df.to_csv('fanfiction_english_records.csv', index=False, sep="|")
print('English total records', len(df))

total records 2067
English total records 1742


# 5. Exporting to HTML

In [11]:
!jupyter nbconvert --output-dir 'htmls/' --to html filtering_english_texts_cld3.ipynb

[NbConvertApp] Converting notebook filtering_english_texts_cld3.ipynb to html
[NbConvertApp] Writing 589411 bytes to htmls/filtering_english_texts_cld3.html
